In [ ]:
# Importing libraries
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import boto3
from datetime import datetime
import scipy
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
# from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from scipy.cluster.vq import kmeans
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# AWS Access Key, Secret Access Key and Session Token (This needs to be updated for each run)
key =    'YOUR_KEY_HERE'
secret = 'YOUR_SECRET_HERE'
token =  'YOUR_TOKEN_HERE'

In [ ]:
# Assigning the values of Bucket names and the raw data file name to variables
raw_data_bucket = 'lloydsbanking'
clean_data_bucket = 'lloydscleandata'
raw_data = 'fake_transactions_set_2.csv'

In [ ]:
# Reading data from CSV file stored in AWS S3 bucket and renaming the column names to follow a particular naming convention
lloyds = pd.read_csv(
    f"s3://{raw_data_bucket}/{raw_data}",
    storage_options={
        "key": 'YOUR_KEY_HERE',
        "secret": 'YOUR_SECRET_HERE',
        "token": 'YOUR_TOKEN_HERE'
    },
)

lloyds = lloyds.rename(columns={'Account Number': 'Account_Number', 'Third Party Account Number' : 'Third_Party_Account_Number', 'Third Party Name' : 'Third_Party_Name'})

In [ ]:
# Displaying the rows and columns of 'lloyds'
lloyds.shape

In [ ]:
# Displaying first 5 rows of the dataframe 'lloyds'
lloyds.head(5)


In [ ]:
# Looping through the 'Date' column of 'lloyds' to modify the year format in the date from "20" to "2020" and updating it
ite = 0
for i in lloyds['Date']:
    tmp = str(i).split('/')
#     print(tmp[2])
    if(tmp[2] == '20'):
        tmp[2] = '2020'
        lloyds['Date'][ite] =  datetime.strptime('/'.join(tmp), '%d/%m/%Y').strftime('%d/%m/%Y')
    ite=ite + 1

In [ ]:
# Displaying the 'Date' column of the DataFrame 'lloyds'
lloyds['Date']

In [ ]:
# Adding a new column 'Type_of_Transaction' to 'lloyds'
lloyds['Type_of_Transaction'] = ''

In [ ]:
# Looping through the 'Amount' column of 'lloyds' 
# Updating the 'Type_of_Transaction' column as 'Debit' if the amount is negative, and 'Credit' if the amount is positive
for i in range(len(lloyds['Amount'])):
    if lloyds['Amount'][i] < 0:
        lloyds['Type_of_Transaction'][i] = 'Debit'
    else:
        lloyds['Type_of_Transaction'][i] = 'Credit'

In [ ]:
# Splitting the 'Date' column of 'lloyds' into separate columns for day, month, and year using the str.split() method
lloyds[["day", "month", "year"]] = lloyds["Date"].str.split("/", expand = True)


In [ ]:
# Updating the 'Third_Party_Account_Number' column of 'lloyds' to 1 where the 'Third_Party_Name' is 'Deliveroo' 
# Updating 2 where the 'Third_Party_Name' is 'Halifax'
lloyds.loc[lloyds['Third_Party_Name'] == 'Deliveroo', 'Third_Party_Account_Number'] = 1
lloyds.loc[lloyds['Third_Party_Name'] == 'Halifax', 'Third_Party_Account_Number'] = 2

In [ ]:
# lloyds.to_csv("/Users/suriyaprakashvadivelu/Documents/Coursework/Mini-Project/lloyds.csv", index = False)

In [ ]:
# Writing the contents of 'lloyds' to a CSV file named 'lloyds_2.csv' 
# Uploading it to the S3 bucket named 'lloydscleandata' using the storage options specified
lloyds.to_csv(
    f"s3://{clean_data_bucket}/lloyds_2.csv",
    index=False,
    storage_options={
        "key": 'YOUR_KEY_HERE',
        "secret": 'YOUR_SECRET_HERE',
        "token": 'YOUR_TOKEN_HERE'
    },
)

# Splitting data into C-C and C-B transactions

In [ ]:
# Creating a new dataframe 'df_customer' by selecting all rows of 'lloyds' where the value in the 'Third_Party_Name' column is null
df_customer = lloyds.loc[lloyds['Third_Party_Name'].isnull()]

In [ ]:
# Displaying rows and columns of 'df_customer'
df_customer.shape

In [ ]:
# Counting the number of occurrences of each unique value in the 'Third_Party_Name' column
df_customer['Third_Party_Name'].value_counts()

In [ ]:
# Converting the 'Third_Party_Account_Number' column of 'df_customer' to integer data type
df_customer['Third_Party_Account_Number'] = df_customer['Third_Party_Account_Number'].astype(int)

In [ ]:
# df_customer.to_csv("/Users/suriyaprakashvadivelu/Documents/Coursework/Mini-Project/cclloyds_2.csv", index = False)

In [ ]:
# Exporting the clean customer data to a new CSV file in the 'split_data' folder of the clean data S3 bucket
df_customer.to_csv(
    f"s3://{clean_data_bucket}/split_data/cclloyds_2.csv",
    index=False,
    storage_options={
        "key": 'YOUR_KEY_HERE',
        "secret": 'YOUR_SECRET_HERE',
        "token": 'YOUR_TOKEN_HERE'
    },
)

In [ ]:
# Creating a new dataframe df_merch by selecting all rows from the lloyds dataframe that are not in the 'df_customer' index
df_merch = lloyds.loc[~lloyds.index.isin(df_customer.index)]

In [ ]:
# Converting the 'Third_Party_Account_Number' column of 'df_merch' to integer data type
df_merch['Third_Party_Account_Number'] = df_merch['Third_Party_Account_Number'].astype(int)

In [ ]:
# df_merch.to_csv("/Users/suriyaprakashvadivelu/Documents/Coursework/Mini-Project/cblloyds_2.csv", index = False)

In [ ]:
# Exporting the clean merchant data to a new CSV file in the 'split_data' folder of the clean data S3 bucket
df_merch.to_csv(
    f"s3://{clean_data_bucket}/split_data/cblloyds_2.csv",
    index=False,
    storage_options={
        "key": 'YOUR_KEY_HERE',
        "secret": 'YOUR_SECRET_HERE',
        "token": 'YOUR_TOKEN_HERE'
    },
)

In [ ]:
# Displaying rows and columns of 'df_merch'
df_merch.shape

## Pulling data from Database

In [ ]:
# Importing the mysql.connector to enable connection to a MySQL database
import mysql.connector

In [ ]:
# Establishing connection with MySQL database using provided credantials
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host='lloydsbanking.cu5hh9wuea2p.us-east-1.rds.amazonaws.com',
                                         database='lloyds_banking',
                                         user='admin',
                                         password='latchu1234')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)


In [ ]:
# Retrieving all customer records from lloyds banking database and converting them into a Pandas dataframe 
cursor.execute("Select * from lloyds_banking.customer_2")
cus = cursor.fetchall()
customer = pd.DataFrame(cus, columns=[x[0] for x in cursor.description])

In [ ]:
# Retrieving all customer group records from lloyds banking database and converting them into a Pandas dataframe.
cursor.execute("Select * from lloyds_banking.cus_group_2")
cus_to_cus = cursor.fetchall()
cus_group = pd.DataFrame(cus_to_cus, columns=[x[0] for x in cursor.description])

In [ ]:
# Retrieving all business group records from lloyds banking database and converting them into a Pandas dataframe.
cursor.execute("Select * from lloyds_banking.bus_group_2")
cus_to_bus = cursor.fetchall()
bus_group = pd.DataFrame(cus_to_bus, columns=[x[0] for x in cursor.description])

In [ ]:
# Retrieving all business group monthly records from lloyds banking database and converting them into a Pandas dataframe.
cursor.execute("Select * from lloyds_banking.bus_group_mon_2")
bus_to_bus_mon = cursor.fetchall()
bus_group_mon = pd.DataFrame(bus_to_bus_mon, columns=[x[0] for x in cursor.description])

In [ ]:
# Retrieving all customer group monthly records from lloyds banking database and converting them into a Pandas dataframe.
cursor.execute("Select * from lloyds_banking.cus_group_mon_2")
cus_to_bus_mon = cursor.fetchall()
cus_group_mon = pd.DataFrame(cus_to_bus_mon, columns=[x[0] for x in cursor.description])

In [ ]:
# Calculating the total yearly transactions for each account number in the bus_group_2 table 
# Storing the result in a Pandas dataframe called bus_group_tot
cursor.execute("Select Account_Number, sum(Total_Amount) as Total_Yearly_Transaction from lloyds_banking.bus_group_2 group by 1")
bus_to_bus_tot = cursor.fetchall()
bus_group_tot = pd.DataFrame(bus_to_bus_tot, columns=[x[0] for x in cursor.description])

In [ ]:
# Calculating the total yearly transactions for each account number in the cus_group_2 table 
# Storing the result in a Pandas dataframe called cus_group_tot
cursor.execute("Select Account_Number, sum(Total_Amount) as Total_Yearly_Transaction from lloyds_banking.cus_group_2 group by 1")
cus_to_bus_mon = cursor.fetchall()
cus_group_tot = pd.DataFrame(cus_to_bus_mon, columns=[x[0] for x in cursor.description])

In [ ]:
# Executing an SQL query to fetch the total amount of money spent by each customer on a monthly basis
# Storing the result it in a list, and converts the list to a pandas dataframe with appropriate column names
cursor.execute("select Account_Number, month_, sum(Amount) from cclloyds_2 group by 1,2")
cus_to_cus_mon_tot = cursor.fetchall()
cus_group_mon_tot = pd.DataFrame(cus_to_cus_mon_tot, columns=[x[0] for x in cursor.description])

In [ ]:
# Executing an SQL query to fetch the total amount of transactions per month and account number in 'cblloyds_2' 
# Storing results in 'bus_group_mon_tot' dataframe
cursor.execute("select Account_Number, month_, sum(Amount) from cblloyds_2 group by 1,2")
bus_to_bus_mon_tot = cursor.fetchall()
bus_group_mon_tot = pd.DataFrame(bus_to_bus_mon_tot, columns=[x[0] for x in cursor.description])

In [ ]:
# Executing SQL query to group lloyds transactions by account number, third party account number and third party name 
# Storing the results in 'lloyds_group'
cursor.execute("select Account_Number,Third_Party_Account_Number,Third_Party_Name,sum(Amount) as Total_Amount,count(Amount) as Number_of_Transaction from lloyds_2 group by 1,2,3")
lloyds_group = cursor.fetchall()
lloyds_group = pd.DataFrame(lloyds_group, columns=[x[0] for x in cursor.description])

In [ ]:
# Executing SQL query to group lloyds transactions by account number, third party account number, third party name and month 
# Storing the results in 'lloyds_group_mon'
cursor.execute("select Account_Number,Third_Party_Account_Number,Third_Party_Name,month_,sum(Amount) as Total_Amount,count(Amount) as Number_of_Transaction from lloyds_2 group by 1,2,3,4")
lloyds_group_mon = cursor.fetchall()
lloyds_group_mon = pd.DataFrame(lloyds_group_mon, columns=[x[0] for x in cursor.description])

In [ ]:
# Retrieving all data from the lloyds table and saves it as a pandas dataframe called 'lloyds_db'
cursor.execute("select * from lloyds_2")
lloyds_db = cursor.fetchall()
lloyds_db = pd.DataFrame(lloyds_db, columns=[x[0] for x in cursor.description])

In [ ]:
# Executing a SQL query to select all rows and columns from 'cclloyds_2' table in the database
# Fetching the results and creating a pandas dataframe named 'ctoc_db'
cursor.execute("select * from cclloyds_2")
ctoc_db = cursor.fetchall()
ctoc_db = pd.DataFrame(ctoc_db, columns=[x[0] for x in cursor.description])

In [ ]:
# Executing a SQL query to select all rows and columns from 'cblloyds_2' table in the database
# Fetching the results and creating a pandas dataframe named 'ctob_db'
cursor.execute("select * from cblloyds_2")
ctob_db = cursor.fetchall()
ctob_db = pd.DataFrame(ctob_db, columns=[x[0] for x in cursor.description])

# EDA

In [ ]:
# Printing 'customer'
customer

### We have 480 individual customers

In [ ]:
# Printing 'ctoc_db'
ctoc_db

In [ ]:
# Getting unique values in the 'year_' column of the 'ctoc_db' dataframe
ctoc_db.year_.unique()

In [ ]:
# Replacing the value '20' in the 'year_' column of the 'ctoc_db' dataframe with '2020'
ctoc_db['year_'] = ctoc_db['year_'].replace(20, 2020)

In [ ]:
# Displaying rows and columns of 'ctob_db'
ctoc_db.shape

In [ ]:
# Getting the top 10 most frequent account numbers in 'ctoc_db'
top_10_acc = ctoc_db.Account_Number.value_counts().nlargest(10)

In [ ]:
# Creating a barplot using seaborn with the top 10 account numbers and their transaction counts
sns.set(rc={'figure.figsize':(20,8.27)})
ax = sns.barplot(x=top_10_acc.index, y=top_10_acc.values, palette="Accent_r")
ax.set(xlabel='Account Numbers', ylabel='No. of Transactions')
plt.show()

In [ ]:
# Selecting the top 10 receivers from the 'ctoc_db'
top_10_receiver = ctoc_db.Third_Party_Account_Number.value_counts().nlargest(10)

In [ ]:
# Creating a barplot using seaborn with the top 10 receiving account numbers and their transaction counts
sns.set(rc={'figure.figsize':(20,8.27)})
ax = sns.barplot(x=top_10_receiver.index, y=top_10_receiver.values, palette="Accent_r")
ax.set(xlabel='Receiving Account Numbers', ylabel='No. of Transactions')
plt.show()

In [ ]:
# Displaying first 5 rows of 'ctoc_db'
ctoc_db.head()

In [ ]:
# Finding the average transactions made in C-C transactions
print('Average C-C transactions: ',ctoc_db.Amount.mean())

# Finding the minimum transaction made in C-C transactions
print('Minimum C-C transactions: ',ctoc_db.Amount.min())

# Finding the maximum transaction made in C-C transactions
print('Maximum C-C transactions: ',ctoc_db.Amount.max())

In [ ]:
# Creating a line plot using Seaborn to visualize the trend of transaction amounts over months
sns.lineplot(data=ctoc_db, x="month_", y="Amount")

In [ ]:
# Creating a countplot using Seaborn to show the frequency of each transaction type in the 'ctoc_db'
sns.countplot(x=ctoc_db["Type_of_Transaction"])

### We can clearly see from the visualization that we have almost equal debit and credit

In [ ]:
# Printing 'cus_group'
cus_group

In [ ]:
# Generating a histogram of the total transaction amounts for customer-to-customer transactions
plt.hist(cus_group['Total_Amount'], bins=20)
plt.xlabel('Total Amount')
plt.ylabel('Frequency')
plt.title('Histogram of Total Transaction Amounts')
plt.show()

## We can clearly see from the visualization that most amount of transactions are around 0

In [ ]:
# Group by third-party account number and sum the number of transactions for each account
top_10 = cus_group.groupby('Third_Party_Account_Number')['Number_of_Transaction'].sum().sort_values(ascending=False)[:10]

# Creating a barplot to show the top 10 most transacted third-party accounts
plt.figure(figsize=(10, 6))
sns.barplot(x=top_10.index, y=top_10.values)
plt.xlabel('Third-Party Account Number')
plt.ylabel('Number of Transactions')
plt.title('Top 10 Most Transacted Third-Party Accounts')
plt.show()

## Account number 1036 has highest number of transactions

In [ ]:
# Creating a scatter plot to visualize the relationship between Number_of_Transaction and Total_Amount
# Using 'Account_Number' variable to color code the points
sns.scatterplot(x="Number_of_Transaction", y="Total_Amount", hue="Account_Number", data=cus_group)
plt.title("Number of Transactions vs. Total Amount by Account Number")
plt.xlabel("Number of Transactions")
plt.ylabel("Total Amount")
plt.show()

In [ ]:
# Displaying top 5 rows of 'cus_group_mon'
cus_group_mon.head()

In [ ]:
# Distribution of total amount by month
sns.boxplot(x='month_', y='Total_Amount', data=cus_group_mon)
plt.title('Distribution of Total Amount by Month')
plt.xlabel('Month')
plt.ylabel('Total Amount')
plt.show()

### The data is very broadly spread around as the total amount varies vastly

In [ ]:
# Printing 'cus_group_tot'
cus_group_tot

In [ ]:
# Grouping the data by 'Account_Number' and calculating the sum of 'Total_Yearly_Transaction'
grouped_data = cus_group_tot.groupby('Account_Number')['Total_Yearly_Transaction'].sum()

# Plotting a bar chart of the 'Total_Yearly_Transaction' for each Account Number
plt.bar(grouped_data.index, grouped_data.values)
plt.xlabel('Account Number')
plt.ylabel('Total Yearly Transaction')
plt.title('Total Yearly Transaction per Account Number')
plt.show()

In [ ]:
# Printing 'cus_group_mon_tot'
cus_group_mon_tot

In [ ]:
# Creating a line plot of total transaction amount over time for each account
plt.figure(figsize=(12, 6))
sns.lineplot(data=cus_group_mon_tot, x='month_', y='sum(Amount)', hue='Account_Number', palette='bright')
plt.title('Total transaction amount over time for each account')
plt.xlabel('Month')
plt.ylabel('Total amount')
plt.show()

### Most of the accounts have lesser amount spent as months pass by

In [ ]:
# Heatmap of total transaction amount by account and month
pivot_table = cus_group_mon_tot.pivot_table(index='Account_Number', columns='month_', values='sum(Amount)')
plt.figure(figsize=(12, 6))
sns.heatmap(pivot_table, cmap='coolwarm', annot=True, fmt='.0f')
plt.title('Total transaction amount by account and month')
plt.xlabel('Month')
plt.ylabel('Account number')
plt.show()

## We can barely see red color since there aren't many positive amount transactions

In [ ]:
# Filtering the dataframe to only include rows for a specific account number
acct_df = cus_group_mon_tot[cus_group_mon_tot['Account_Number'] == 1036]

# Grouping the dataframe by month and sum the Amount
grouped_df = acct_df.groupby('month_')['sum(Amount)'].sum()

# Plotting a line chart of the monthly sum of amount for the account number
plt.plot(grouped_df.index, grouped_df.values)
plt.xlabel('Month')
plt.ylabel('Monthly Sum of Amount')
plt.title('Monthly Sum of Amount for Account Number 1036')
plt.show()

### The Monthly sum is lesser as the number of months pass by

# Customer - Business Transactions EDA

In [ ]:
# Displaying top 5 rows of 'ctob_db'
ctob_db.head()

In [ ]:
# Selecting the top 10 third-party names with the most transactions
top_10_bus = ctob_db.Third_Party_Name.value_counts().nlargest(10)

In [ ]:
# Creating a barplot using seaborn with popular businesses and their transaction counts
sns.set(rc={'figure.figsize':(20,8.27)})
ax = sns.barplot(x=top_10_bus.index, y=top_10_bus.values, palette="Accent_r")
ax.set(xlabel='Popular Businesses', ylabel='No. of Transactions')
plt.title('Top 10 Popular businesses')
plt.show()

## Co-op local is the most popular business amongst the transactions 

In [ ]:
# Finding the average transactions made in C-B transactions
print('Average C-B transactions: ',ctob_db.Amount.mean())

# Finding the minimum transactions made in C-B transactions
print('Minimum C-B transactions: ',ctob_db.Amount.min())

# Finding the maximum transactions made in C-B transactions
print('Maximum C-B transactions: ',ctob_db.Amount.max())

## The mean amount of transactions is around 10 pounds. It is -ve since it is a debit

In [ ]:
# Creating a line plot using Seaborn to visualize the trend of transaction amounts over months
sns.lineplot(data=ctob_db, x="month_", y="Amount")

## Most money is transacted around March

In [ ]:
# Creating a line plot using Seaborn to visualize the trend of transaction amounts over each days of the month
sns.lineplot(data=ctob_db, x="day_", y="Amount")

# Through the visualization we can see that towards end of the month the amount of transactions gets increased

In [ ]:
# Creating a barplot that shows the count of transactions on each month of 'ctob_db'
sns.countplot(x=ctob_db["month_"])

# It is very evident that first 3 months have highest number of transactions

In [ ]:
# Creating a countplot using Seaborn to show the frequency of each transaction type in the 'ctoc_db'
sns.countplot(x=ctob_db["Type_of_Transaction"])

### Not like C-C in C-B transactions the debit is more when compared to credit

In [ ]:
# Displaying first 5 rows of 'ctob_db'
ctob_db.head()

In [ ]:
# Grouping the data by 'Account_Number' and counting the frequency of each group
account_counts = ctob_db.groupby('Account_Number').size().reset_index(name='Frequency')

# Sorting the groups in descending order based on frequency and selecting the top 50
top_50_accounts = account_counts.sort_values(by='Frequency', ascending=False).head(50)

# Displaying the top 50 accounts
print(top_50_accounts)

## Finding the top 20 accounts which have the highest frequency of transactions

In [ ]:
# Creating a bar plot of top 50 frequently transacted account numbers
plt.bar(top_50_accounts['Account_Number'], top_50_accounts['Frequency'])
plt.xlabel('Account Number')
plt.ylabel('Frequency')
plt.title('Top 50 Most Frequently Transacted Account Numbers')
plt.show()

## Plotting the most frequent transcating accounts

In [ ]:
# Creating a scatterplot of Amount vs Balance using the data from 'ctob_db'
sns.scatterplot(x='Amount', y='Balance', data=ctob_db)
plt.title('Amount vs Balance')
plt.show()

## It is evident that the people who have high balance spend more for each transaction

## The customers who have negative balance do lesser transactions with lesser amounts

In [ ]:
# Grouping the data by month and transaction type and counting the transactions
transactions_by_month = ctob_db.groupby(['month_', 'Type_of_Transaction'])['Amount'].count().reset_index()

# Pivot the data to create a stacked bar chart with a separate bar for each transaction type
transactions_by_month = transactions_by_month.pivot(index='month_', columns='Type_of_Transaction', values='Amount')

# Creating a stacked bar chart of the transactions by month
transactions_by_month.plot(kind='bar', stacked=True, figsize=(10,6))
plt.xlabel('Month')
plt.ylabel('Number of Transactions')
plt.title('Transactions by Month and Type')
plt.show()

In [ ]:
# Grouping the data by third-party name and calculating the total number of transactions for each name
transactions_by_name = bus_group.groupby('Third_Party_Name')['Number_of_Transaction'].sum().reset_index()

# Creating a bar chart of the number of transactions by third-party name
plt.figure(figsize=(10,6))
plt.bar(transactions_by_name['Third_Party_Name'], transactions_by_name['Number_of_Transaction'])
plt.xticks(rotation=90)
plt.xlabel('Third-Party Name')
plt.ylabel('Number of Transactions')
plt.title('Number of Transactions by Third-Party Name')
plt.show()

## Depecting all the businesses and the number of transactions made to each

In [ ]:
# Creating a scatter plot of the number of transactions vs the total amount for each third-party business
plt.figure(figsize=(10,6))
plt.scatter(bus_group['Number_of_Transaction'], bus_group['Total_Amount'])
plt.xlabel('Number of Transactions')
plt.ylabel('Total Amount')
plt.title('Total Amount vs. Number of Transactions')
plt.show()

## As the number of transactions increase, the total amout debited from the account increases which is why the total amount is in the negative. We also notice that when the number of transactions is less frequent the amount is also less. When it is credit the number of trasanctions are very less since it might be a monthly transcation which indicates a salary credit since it is C-B transactions

In [ ]:
# Printing 'bus_group_mon'
bus_group_mon

In [ ]:
# Aggregating data by Third Party Name and calculate total amount
total_by_third_party = bus_group.groupby("Third_Party_Name")["Total_Amount"].sum()

# Creating bar plot of the total amount transacted by each third party name
plt.bar(total_by_third_party.index, total_by_third_party.values)
plt.xticks(rotation=90)
plt.title("Total Amount by Third Party Name")
plt.xlabel("Third Party Name")
plt.ylabel("Total Amount")
plt.show()

### Here we see that co-op local has the maximum total amount in the negative side, so that means money is credited and that the maxmimum amount of credit has happend for co-op. For business like Red Lion, Rose & Crown, The Crown, The Royal Oak, White Hart have positive values which implies that the money has been credited to the customer. This implies that this is either a cash repayment or salary given to the accounts

In [ ]:
# Creating a scatter plot of the total amount vs number of transactions for businesses by month
plt.scatter(bus_group_mon["Number_of_Transaction"], bus_group_mon["Total_Amount"])
plt.title("Total Amount vs Number of Transactions")
plt.xlabel("Number of Transactions")
plt.ylabel("Total Amount")
plt.show()

## Calculating Salary, Expenditure, Savings and End of month balance for each account

In [ ]:
# Executing the SQL query to retrieve salary data from the database and converting them to a pandas dataframe 'sal_df'
cursor.execute("select Account_Number, Third_Party_Name, month_, Salary from (select Account_Number, Third_Party_Name, month_,sum(if(Amount > 0, Amount, 0)) as Salary from lloyds_2 group by 1,2,3 having Third_Party_Name != '') a where Salary > 0")
sal_df = cursor.fetchall()
sal_df = pd.DataFrame(sal_df, columns=[x[0] for x in cursor.description])

In [ ]:
# Execute SQL query to retrieve account number, month, and expenditure data and converting them to a pandas dataframe 'exp_df'
cursor.execute("select Account_Number, month_, sum(ABS(if(Amount < 0, Amount, 0))) as Expenditure from lloyds_2 group by 1,2")
exp_df = cursor.fetchall()
exp_df = pd.DataFrame(exp_df, columns=[x[0] for x in cursor.description])

In [ ]:
# Executing SQL query to fetch data related to salary, expenditure, and savings from the database and converting the results into pandas dataframe 'sav_df'
cursor.execute("select sal.Account_Number, sal.month_, (Salary - Expenditure) as Savings from (select Account_Number, Third_Party_Name, month_, Salary from (select Account_Number, Third_Party_Name, month_,sum(if(Amount > 0, Amount, 0)) as Salary from lloyds_2 group by 1,2,3 having Third_Party_Name != '') a where Salary > 0) sal left join (select Account_Number, month_, sum(ABS(if(Amount < 0, Amount, 0))) as Expenditure from lloyds_2 group by 1,2) expe on sal.Account_Number = expe.Account_Number group by 1,2")
sav_df = cursor.fetchall()
sav_df = pd.DataFrame(sav_df, columns=[x[0] for x in cursor.description])

In [ ]:
# Executing SQL query to select Account Number, month and end-of-month Balance from Lloyds table and converting them into pandas dataframe 'eom_bal_df'
cursor.execute("select Account_Number, month_, Balance from (select Account_Number, month_, Balance, row_number() OVER(PARTITION BY Account_Number,month_ ORDER BY Date DESC, Time Desc) as row_num from lloyds_2) rn where row_num = 1")
eom_bal_df = cursor.fetchall()
eom_bal_df = pd.DataFrame(eom_bal_df, columns=[x[0] for x in cursor.description])

In [ ]:
# Group 'sal_df' by 'Third_Party_Name' and 'month_' to get the total salary for each combination of third party and month
df_grouped = sal_df.groupby(['Third_Party_Name', 'month_'])['Salary'].sum().reset_index()

# Looping over each unique third party name in 'df_grouped' and plot a line for their salary by month
for name in df_grouped['Third_Party_Name'].unique():
    data = df_grouped[df_grouped['Third_Party_Name'] == name]
    plt.plot(data['month_'], data['Salary'], label=name)
plt.legend()
plt.xlabel('Month')
plt.ylabel('Salary')
plt.title('Salary by Third Party')
plt.show()


In [ ]:
# Create a scatter plot
plt.scatter(sal_df['Account_Number'], sal_df['Salary'])

# Set the plot title and axes labels
plt.title('Salary vs. Account Number')
plt.xlabel('Account Number')
plt.ylabel('Salary')

# Display the plot
plt.show()

In [ ]:
# Aggregating data by 'Third_Party_Name' and calculating total amount
total_by_third_party = sal_df.groupby("Third_Party_Name")["Salary"].sum()

# Creating bar plot showing the total salary received from each third-party in 'sal_df'
plt.bar(total_by_third_party.index, total_by_third_party.values)
plt.xticks(rotation=90)
plt.title("Total Amount by Third Party Name")
plt.xlabel("Third Party Name")
plt.ylabel("Total Amount")
plt.show()

In [ ]:
# Printing 'eom_bal_df'
eom_bal_df

In [ ]:
# Creating a box plot of the balance for each month
sns.boxplot(x="month_", y="Balance", data=eom_bal_df)

# Adding axis labels
plt.xlabel("Month")
plt.ylabel("Balance")
plt.show()

In [ ]:
# Sorting the data by balance in ascending order and selecting the top 5 accounts
top_negative = eom_bal_df.sort_values(by='Balance').head(10)
print("Top 5 accounts with the lowest balance:")
print(top_negative)

# Sorting the data by balance in descending order and selecting the top 5 accounts
top_positive = eom_bal_df.sort_values(by='Balance', ascending=False).head(10)
print("\nTop 5 accounts with the highest balance:")
print(top_positive)

# Creating a figure with two subplots
fig, ax = plt.subplots(nrows=2, figsize=(10, 8))

# Plotting the top 5 accounts with lowest balance on this subplot
ax[0].bar(top_negative['Account_Number'], top_negative['Balance'])
ax[0].set_title('Top 5 Accounts with Lowest Balance')
ax[0].set_xlabel('Account Number')
ax[0].set_ylabel('Balance')

# Plotting the top 5 accounts with highest balance on this subplot
ax[1].bar(top_positive['Account_Number'], top_positive['Balance'])
ax[1].set_title('Top 5 Accounts with Highest Balance')
ax[1].set_xlabel('Account Number')
ax[1].set_ylabel('Balance')

# Adjusting the layout and displaying the plot
plt.tight_layout()
plt.show()

## As you can see there are outliers and March month has most number of outliers

## Overall Customer Segmentation

In [ ]:
# Selecting 'Number_of_Transaction' and 'Total_Amount' columns from the lloyds_group
col_names = ['Number_of_Transaction', 'Total_Amount']
features = lloyds_group[col_names]
# new_features = pd.DataFrame(features, columns = col_names)

# Standarizing the values and storing the results in scaled_features
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = col_names)

In [ ]:
# Performing KMeans clustering on the selected standardized features for different number of clusters
# Calculating the sum of squared errors (SSE) for each cluster
SSE = []
for cluster in range(1,15):
    kmeans = KMeans(n_clusters = cluster, init='k-means++')
    kmeans.fit(scaled_features)
    SSE.append(kmeans.inertia_)
# Converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,15), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
# Performing KMeans clustering using 4 clusters, and storing the result in 'kmeans'
kmeans = KMeans( n_clusters = 4, init='k-means++')
kmeans.fit(scaled_features)

In [ ]:
# Calculating the Silhouette score of the KMeans clustering for labels generated by KMeans and the Euclidean distance metric
print(silhouette_score(scaled_features, kmeans.labels_, metric='euclidean'))


In [ ]:
# Predicting the cluster labels for each row and adding the result as a new column 'cluster' in a new dataframe called 'frame'
pred = kmeans.predict(scaled_features)
frame = pd.DataFrame(lloyds_group)
frame['cluster'] = pred

In [ ]:
# Displaying the number of occurrences of each cluster label in the 'cluster' column of the 'frame'
frame.cluster.value_counts()

In [ ]:
#x_axis = frame['Number_of_Transaction']
#y_axis = frame['Total_Amount']

# Creating a scatter plot for visualzing the clustering results
plt.figure(figsize = (10, 8))
sns.scatterplot(data=frame, x='Number_of_Transaction', y='Total_Amount', hue=frame['cluster'])
plt.title('Segmentation K-means')
plt.show()

## Monthly Segmentation

In [ ]:
# Printing 'lloyds_group_mon'
lloyds_group_mon

In [ ]:
# Selecting 'Number_of_Transaction' and 'Total_Amount' columns from the lloyds_group_mon
col_names = ['Number_of_Transaction', 'Total_Amount']
features = lloyds_group_mon[col_names]

# Standarizing the values and storing the results in scaled_features
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = col_names)

In [ ]:
# Performing KMeans clustering on the selected standardized features for different number of clusters
# Calculating the sum of squared errors (SSE) for each cluster
SSE = []
for cluster in range(1,15):
    kmeans = KMeans(n_clusters = cluster, init='k-means++')
    kmeans.fit(scaled_features)
    SSE.append(kmeans.inertia_)
# converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,15), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
# Performing KMeans clustering using 3 clusters, and storing the result in 'kmeans'
kmeans = KMeans( n_clusters = 3, init='k-means++')
kmeans.fit(scaled_features)

In [ ]:
# Calculating the Silhouette score of the KMeans clustering for labels generated by KMeans and the Euclidean distance metric
print(silhouette_score(scaled_features, kmeans.labels_, metric='euclidean'))

In [ ]:
# Predicting the cluster labels for each row and adding the result as a new column 'cluster' in a new dataframe called 'frame'
pred = kmeans.predict(scaled_features)
frame = pd.DataFrame(lloyds_group_mon)
frame['cluster'] = pred

In [ ]:
# Displaying the number of occurrences of each cluster label in the 'cluster' column of the 'frame'
frame.cluster.value_counts()

In [ ]:
# df_segm_analysis = frame.groupby(['cluster']).mean()
# df_segm_analysis

In [ ]:
#x_axis = frame['Number_of_Transaction']
#y_axis = frame['Total_Amount']

# Creating a scatter plot for visualzing the clustering results
plt.figure(figsize = (10, 8))
sns.scatterplot(data=frame, x='Number_of_Transaction', y='Total_Amount', hue=frame['cluster'])
plt.title('Segmentation K-means')
plt.show()

## Customer segmentation based on No. of transactions and amount of transactions

In [ ]:
# Printing 'cus_group'
cus_group

In [ ]:
# Selecting 'Number_of_Transaction' and 'Total_Amount' columns from the cus_group
col_names = ['Number_of_Transaction', 'Total_Amount']
features = cus_group[col_names]

# Standarizing the values and storing the results in scaled_features
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = col_names)

In [ ]:
# Performing KMeans clustering on the selected standardized features for different number of clusters
# Calculating the sum of squared errors (SSE) for each cluster
SSE = []
for cluster in range(1,15):
    kmeans = KMeans(n_clusters = cluster, init='k-means++')
    kmeans.fit(scaled_features)
    SSE.append(kmeans.inertia_)
# converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,15), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
# Performing KMeans clustering using 3 clusters, and storing the result in 'kmeans'
kmeans = KMeans( n_clusters = 3, init='k-means++')
kmeans.fit(scaled_features)

In [ ]:
# Calculating the Silhouette score of the KMeans clustering for labels generated by KMeans and the Euclidean distance metric
print(silhouette_score(scaled_features, kmeans.labels_, metric='euclidean'))

In [ ]:
# Predicting the cluster labels for each row and adding the result as a new column 'cluster' in a new dataframe called 'frame'
pred = kmeans.predict(scaled_features)
frame = pd.DataFrame(cus_group)
frame['cluster'] = pred

In [ ]:
# Displaying first five rows for 'frame'
frame.head()

In [ ]:
# Displaying the number of occurrences of each cluster label in the 'cluster' column of the 'frame'
frame.cluster.value_counts()

In [ ]:
# Grouping the dataframe 'frame' by the 'cluster' column and calculating mean for each column and stored in 'df_segm_analysis'
df_segm_analysis = frame.groupby(['cluster']).mean()
df_segm_analysis

In [ ]:
# Renaming cluster label into appropraite names 
df_segm_analysis.rename({0:'Less-Frequent and Amount Transactors',
                         1:'More Frequent and Amount Transactors',
                         })

In [ ]:
# Printing 'frame'
frame

In [ ]:
#x_axis = frame['Number_of_Transaction']
#y_axis = frame['Total_Amount']

# Creating a scatter plot for visualzing the clustering results
plt.figure(figsize = (10, 8))
sns.scatterplot(data=frame, x='Number_of_Transaction', y='Total_Amount', hue=frame['cluster'])
plt.title('Segmentation K-means')
plt.show()

In [ ]:
# Printing 'bus_group'
bus_group

In [ ]:
# Selecting 'Number_of_Transaction' and 'Total_Amount' columns from the bus_group
col_names = ['Number_of_Transaction', 'Total_Amount']
features2 = bus_group[col_names]

# Standarizing the values and storing the results in scaled_features2
scaler2 = StandardScaler().fit(features2.values)
features2 = scaler.transform(features2.values)
scaled_features2 = pd.DataFrame(features2, columns = col_names)

In [ ]:
# Printing 'scaled_features2'
scaled_features2

In [ ]:
# Performing KMeans clustering on the selected standardized features for different number of clusters
# Calculating the sum of squared errors (SSE) for each cluster
SSE = []
for cluster in range(1,10):
    kmeans = KMeans(n_clusters = cluster, init='k-means++')
    kmeans.fit(scaled_features2)
    SSE.append(kmeans.inertia_)

# Converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,10), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
# Performing KMeans clustering using 3 clusters, and storing the result in 'kmeans2'
kmeans2 = KMeans( n_clusters = 3, init='k-means++')
kmeans2.fit(scaled_features2)

In [ ]:
# Calculating the Silhouette score of the KMeans clustering for labels generated by KMeans and the Euclidean distance metric
print(silhouette_score(scaled_features2, kmeans2.labels_, metric='euclidean'))

In [ ]:
# Predicting the cluster labels for each row and adding the result as a new column 'cluster' in a new dataframe called 'frame2'
pred2 = kmeans2.predict(scaled_features2)
frame2 = pd.DataFrame(bus_group)
frame2['cluster'] = pred2

In [ ]:
# Printing 'frame2'
frame2

In [ ]:
# Displaying the number of occurrences of each cluster label in the 'cluster' column of the 'frame2'
frame2.cluster.value_counts()

In [ ]:
# Creating a scatter plot for visualzing the clustering results
x_axis = frame2['Number_of_Transaction']
y_axis = frame2['Total_Amount']
plt.figure(figsize = (10, 8))
sns.scatterplot(data=frame2, x='Number_of_Transaction', y='Total_Amount', hue=frame2['cluster'])
plt.title('Segmentation K-means')
plt.show()

In [ ]:
# Counting the number of occurrences of each unique value in the 'month_' column of 'bus_group_mon'
bus_group_mon['month_'].value_counts()

In [ ]:
# Counting the number of occurrences of each unique value in the 'Third_Party_Name' column of 'bus_group_mon'
bus_group_mon['Third_Party_Name'].value_counts()

In [ ]:
# Using For loop to loop through each month from Jan to Dec and filtering the data to include only transactions in that month
for month in range(1, 13):
    df_mon = bus_group_mon[bus_group_mon["month_"] == month]

    # Grouping the data by 'from_totally_fake_account' and calculating the sum of 'Number_of_Transaction'
    df_mon_grouped = df_mon.groupby("Third_Party_Name")["Number_of_Transaction"].sum()

    # Sorting the data in descending order based on the sum of 'Number_of_Transaction'
    df_mon_grouped_sorted = df_mon_grouped.sort_values(ascending=False)

    # Plotting a bar chart of the top 10 'From Accounts' with maximum transactions in January
    df_mon_grouped_sorted.head(10).plot(kind="bar")
    plt.xlabel("From Account")
    plt.ylabel("Number of Transactions")
    plt.title("Top 10 From Accounts with Maximum Transactions in month {}".format(month))
    plt.show()

In [ ]:
# Printing 'eom_bal_df'
eom_bal_df

In [ ]:
# Selecting 'Balance'column from the eom_bal_df 
col_names = ['Balance']
features = eom_bal_df[col_names]

# Standarizing the values and storing the results in scaled_features
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = col_names)

In [ ]:
# Performing KMeans clustering on the selected standardized features for different number of clusters
# Calculating the sum of squared errors (SSE) for each cluster
SSE = []
for cluster in range(1,15):
    kmeans = KMeans(n_clusters = cluster, init='k-means++')
    kmeans.fit(scaled_features)
    SSE.append(kmeans.inertia_)
# Converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,15), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
# Performing KMeans clustering using 4 clusters, and storing the result in 'kmeans'
kmeans = KMeans( n_clusters = 4, init='k-means++')
kmeans.fit(scaled_features)

In [ ]:
# Calculating the Silhouette score of the KMeans clustering for labels generated by KMeans and the Euclidean distance metric
print(silhouette_score(scaled_features, kmeans.labels_, metric='euclidean'))

In [ ]:
# Predicting the cluster labels for each row and adding the result as a new column 'cluster' in a new dataframe called 'frame'
pred = kmeans.predict(scaled_features)
frame = pd.DataFrame(eom_bal_df)
frame['cluster'] = pred

In [ ]:
# Displaying the number of occurrences of each cluster label in the 'cluster' column of the 'frame'
frame.cluster.value_counts()

In [ ]:
# Grouping the dataframe 'frame' by the 'cluster' column and calculating mean for each column and stored in 'df_segm_analysis'
df_segm_analysis = frame.groupby(['cluster']).mean()
df_segm_analysis

In [ ]:
# Creating a scatter plot for visualzing the clustering results
plt.figure(figsize = (10, 8))
sns.scatterplot(data=frame, x='Balance', y='Account_Number', hue=frame['cluster'])
plt.title('Segmentation K-means')
plt.show()

# Anomaly detection

In [ ]:
# Printing 'bus_group'
bus_group

In [ ]:
# Creating a box plot to visualize the distribution of 'Total_Amount'
bus_group['Total_Amount'].plot(kind='box')
plt.show()

In [ ]:
# Creating a histogram plot to visualize the distribution of 'Total_Amount'
bus_group['Total_Amount'].plot(kind='hist')
plt.show()

In [ ]:
# Printing the minimum and maximum amount in the 'Total_Amount' column of the 'bus_group' dataframe
print('Minimum Amount ' + str(bus_group['Total_Amount'].min()))
print('Maximum Amount ' + str(bus_group['Total_Amount'].max()))

In [ ]:
# Reshaping the 'Total_Amount' column to fit the input shape
amount_raw = bus_group['Total_Amount'].values

# For compatibility with the SciPy implementation
amount_raw = amount_raw.reshape(-1, 1)
amount_raw = amount_raw.astype('float64')

In [ ]:
# Specifying the data and the number of clusters to kmeans()
# centroids, avg_distance = kmeans(amount_raw, 4)

In [ ]:
# Grouping the data points into clusters using k-means algorithm
# vq function from scipy.cluster.vq library is used to group the data into clusters based on the centroids
# groups, cdist = scipy.cluster.vq.vq(amount_raw, centroids)

In [ ]:
# Printing length of 'amount_raw'
# len(amount_raw)

In [ ]:
# # Plotting the total amount values against their corresponding indices
# plt.scatter(amount_raw, np.arange(0,6177), c=groups)
# plt.xlabel('Total Amount')
# plt.ylabel('Indices')
# plt.show()

## Anomalies detected. These could be fraudulent transactions

# Anomaly detection using Isolation Forest for C-B transactions

In [ ]:
# Creating a new variable 'newBus'
newBus = bus_group

In [ ]:
# Droping columns that won't be used for the analysis
newBus = newBus.drop(['Account_Number', 'Third_Party_Account_Number', 'Third_Party_Name'], axis=1)

# Fitting the Isolation Forest model
clf = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(0.01), max_features=1.0)
clf.fit(newBus)

# Predicting the anomalies
anomalies = clf.predict(newBus)

# Adding the anomalies as a column to the original dataframe
newBus['Anomaly'] = anomalies

# Printing the anomalous transactions
print(newBus[newBus['Anomaly'] == -1])

In [ ]:
# Printing 'newBus'
newBus

In [ ]:
# Plotting a scatter plot of the anomalies
plt.scatter(newBus["Number_of_Transaction"], newBus["Total_Amount"], c=newBus["Anomaly"], cmap='seismic_r')
plt.xlabel("Number of Transactions")
plt.ylabel("Total Amount")
plt.title("Anomaly Detection")
plt.show()

In [ ]:
# Column has a value of 1, indicating that the data point is considered normal
normal_df = newBus[newBus['Anomaly'] == 1]

# Column has a value of -1, indicating that the data point is considered an anomaly
anomaly_df = newBus[newBus['Anomaly'] == -1]

In [ ]:
# Creating figure and axis objects
fig, ax = plt.subplots(figsize=(10,6))

# Plotting data points with anomaly classification
ax.scatter(normal_df.index, normal_df['Total_Amount'], color='blue', label='Normal transaction')
ax.scatter(anomaly_df.index, anomaly_df['Total_Amount'], color='red', label='Anomaly transaction')

# Setting title and labels
ax.set_title('Anomaly Detection', fontsize=14)
ax.set_xlabel('Transaction Index', fontsize=12)
ax.set_ylabel('Total Amount', fontsize=12)

# Adding separate legend for anomaly transactions
anomaly_patch = plt.scatter([], [], color='red', label='Anomaly transaction')
normal_patch = plt.scatter([], [], color='blue', label='Normal transaction')
plt.legend(handles=[anomaly_patch, normal_patch], loc='upper left')

# Showing plot
plt.show()

In [ ]:
# Printing 'cus_group'
cus_group

# Anomaly detection for C-C transactions using IsolationForest

In [ ]:
# Creating a new variable 'newCus'
newCus = cus_group

In [ ]:
# Droping columns that won't be used for the analysis
newCus = newCus.drop(['Account_Number', 'Third_Party_Account_Number'], axis=1)

# Fitting the Isolation Forest model
clf = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(0.01), max_features=1.0)
clf.fit(newCus)

# Predicting the anomalies
anomalies = clf.predict(newCus)

# Adding the anomalies as a column to the original dataframe
newCus['Anomaly'] = anomalies

# Printing the anomalous transactions
print(newCus[newCus['Anomaly'] == -1])

In [ ]:
# Column has a value of 1, indicating that the data point is considered normal
normal_df_c = newCus[newCus['Anomaly'] == 1]

# Column has a value of -1, indicating that the data point is considered an anomaly
anomaly_df_c = newCus[newCus['Anomaly'] == -1]

In [ ]:
# Creating figure and axis objects
fig, ax = plt.subplots(figsize=(10,6))

# Ploting data points with anomaly classification
ax.scatter(normal_df_c.index, normal_df_c['Total_Amount'], color='blue', label='Normal transaction')
ax.scatter(anomaly_df_c.index, anomaly_df_c['Total_Amount'], color='red', label='Anomaly transaction')

# Setting title and labels
ax.set_title('Anomaly Detection', fontsize=14)
ax.set_xlabel('Transaction Index', fontsize=12)
ax.set_ylabel('Total Amount', fontsize=12)

# Adding separate legend for anomaly transactions
anomaly_patch = plt.scatter([], [], color='red', label='Anomaly transaction')
normal_patch = plt.scatter([], [], color='blue', label='Normal transaction')
plt.legend(handles=[anomaly_patch, normal_patch], loc='upper left')

# Showing plot
plt.show()

# Customer Churn Prediction based on Number of Transactions C-B


In [ ]:
# Calculating the number of debit transactions for each customer
counts = ctob_db[ctob_db["Type_of_Transaction"] == "Debit"].groupby("Account_Number").size()

# Classifying customers based on number of debit transactions
def classify_customer(count):
    if count < 50:
        return "Definitely Churn"
    elif count < 100 and count >50:
        return "High Chance of Churning"
    else:
        return "Low Chance of Churning"
    
customer_segments = counts.apply(classify_customer)

# Plotting the results
fig, ax = plt.subplots(figsize=(16, 6))

colors = customer_segments.apply(lambda x: {
    "Definitely Churn": "#4287f5",
    "High Chance of Churning": "#81bef7",
    "Low Chance of Churning": "#c8dbf7"
}[x])

customer_segments.value_counts().plot(kind="barh", color=colors)

ax.set_title("Customer Churn Segmentation")
ax.set_xlabel("Number of Customers")
ax.set_ylabel("Churn Segment")


plt.tight_layout()
plt.show()


In [ ]:
# Creating a new dataframe with the count of debit transactions for each account number
counts_df = ctob_db[ctob_db["Type_of_Transaction"] == "Debit"].groupby("Account_Number").size().reset_index(name="Debit_Count")

# Classifying customers based on their debit transaction count
def classify_customers(row):
    count = row["Debit_Count"]
    if count < 50:
        return "High Churn Risk"
    elif count < 100 and count >50:
        return "Medium Churn Risk"
    else:
        return "Low Churn Risk"

counts_df["Churn_Risk"] = counts_df.apply(classify_customers, axis=1)

# Creating a scatter plot with different colors for each segment
colors = {"High Churn Risk": "red", "Medium Churn Risk": "orange", "Low Churn Risk": "green"}
plt.figure(figsize=(12, 8), dpi=80)
plt.scatter(counts_df["Account_Number"], counts_df["Debit_Count"], c=counts_df["Churn_Risk"].apply(lambda x: colors[x]), alpha=0.5)

# Adding labels and title to the plot

plt.xlabel("Account Number")
plt.ylabel("Debit Transaction Count")
plt.title("Customer Segmentation by Debit Transaction Count")

# Showing the plot
plt.show()


In [ ]:
# Creating separate dataframes based on churn risk
high_churn_risk_df = counts_df[counts_df["Churn_Risk"] == "High Churn Risk"]
medium_churn_risk_df = counts_df[counts_df["Churn_Risk"] == "Medium Churn Risk"]
low_churn_risk_df = counts_df[counts_df["Churn_Risk"] == "Low Churn Risk"]


In [ ]:
# Printing 'high_churn_risk_df'
high_churn_risk_df

In [ ]:
# Printing 'medium_churn_risk_df'
medium_churn_risk_df

In [ ]:
# Printing 'ctoc_db'
ctoc_db

In [ ]:
# Filtering for debit transactions and group by account number
debit_counts = ctoc_db[ctoc_db["Type_of_Transaction"] == "Debit"].groupby("Account_Number").size().reset_index(name="Debit_Count")

# Calculating the mean debit count
mean_debit_count = debit_counts["Debit_Count"].mean()

print("Mean number of debit transactions:", mean_debit_count)

In [ ]:
# Creating a new dataframe with the count of debit transactions for each account number
counts = ctoc_db[ctoc_db["Type_of_Transaction"] == "Debit"].groupby("Account_Number").size().reset_index(name="Debit_Count")

# Classifying customers based on their debit transaction count
def classify_customers(row):
    count = row["Debit_Count"]
    if count < 5:
        return "High Churn Risk"
    elif count > 4 and count < 18:
        return "Medium Churn Risk"
    else:
        return "Low Churn Risk"

counts["Churn_Risk"] = counts.apply(classify_customers, axis=1)

# Creating a scatter plot with different colors for each segment
colors = {"High Churn Risk": "red", "Medium Churn Risk": "orange", "Low Churn Risk": "green"}
plt.figure(figsize=(12, 8), dpi=80)
plt.scatter(counts["Account_Number"], counts["Debit_Count"], c=counts["Churn_Risk"].apply(lambda x: colors[x]), alpha=0.5)

# Adding labels and title to the plot

plt.xlabel("Account Number")
plt.ylabel("Debit Transaction Count")
plt.title("Customer Segmentation by Debit Transaction Count")

# Showing the plot
plt.show()


In [ ]:
# Creating separate dataframes based on churn risk
high_churn_risk = counts[counts["Churn_Risk"] == "High Churn Risk"]
medium_churn_risk = counts[counts["Churn_Risk"] == "Medium Churn Risk"]
low_churn_risk = counts[counts["Churn_Risk"] == "Low Churn Risk"]

In [ ]:
# Printing 'high_churn_risk'
high_churn_risk

In [ ]:
lb= lloyds_db.copy()


In [ ]:
# Convert the 'Date' column to a Pandas datetime format
lb['Date'] = pd.to_datetime(lb['Date'])

# Group the data by day and count the number of transactions
transactions_per_day = lb.groupby(['Date'])['Account_Number'].count()

# Calculate the mean, minimum, and maximum number of transactions per day
mean_transactions_per_day = transactions_per_day.mean()
min_transactions_per_day = transactions_per_day.min()
max_transactions_per_day = transactions_per_day.max()

# Print the results
print('Mean number of transactions per day:', mean_transactions_per_day)
print('Minimum number of transactions per day:', min_transactions_per_day)
print('Maximum number of transactions per day:', max_transactions_per_day)

In [ ]:
# Creating a new dataframe with the count of debit transactions for each account number
counts_df = lb[lb["Type_of_Transaction"] == "Debit"].groupby("Account_Number").size().reset_index(name="Debit_Count")

# Classifying customers based on their debit transaction count
def classify_customers(row):
    count = row["Debit_Count"]
    if count < 150:
        return "High Churn Risk"
    elif count < 149 and count >360:
        return "Medium Churn Risk"
    else:
        return "Low Churn Risk"

counts_df["Churn_Risk"] = counts_df.apply(classify_customers, axis=1)

# Creating a scatter plot with different colors for each segment
colors = {"High Churn Risk": "red", "Medium Churn Risk": "orange", "Low Churn Risk": "green"}
plt.figure(figsize=(12, 8), dpi=80)
plt.scatter(counts_df["Account_Number"], counts_df["Debit_Count"], c=counts_df["Churn_Risk"].apply(lambda x: colors[x]), alpha=0.5)

# Adding labels and title to the plot

plt.xlabel("Account Number")
plt.ylabel("Debit Transaction Count")
plt.title("Customer Segmentation by Debit Transaction Count")

# Showing the plot
plt.show()


In [ ]:
counts_df.mean()

In [ ]:
counts_df.min()

In [ ]:
counts_df.max()

In [ ]:
# Checking whether the data types are correct for each column
lloyds_db.info()

# Using regression to predict credit card worthiness of customers based on balance

In [ ]:
# Creating a new variable 'LLDB'
LLDB = lloyds_db.copy()

In [ ]:
# Droping unimportant features
LLDB.drop(["Third_Party_Account_Number", "Third_Party_Name", "Time", "Type_of_Transaction"], axis=1, inplace=True)

# Converting date to datetime format and extract month and year
LLDB['Date'] = pd.to_datetime(LLDB['Date'])
LLDB['month'] = LLDB['Date'].dt.month
LLDB['year'] = LLDB['Date'].dt.year

# Calculating the balance at the end of each month for each customer
balance = LLDB.groupby(['Account_Number', 'month', 'year']).tail(1)[['Account_Number', 'month', 'year', 'Balance']]
balance['creditworthy'] = (balance['Balance'] > 0).astype(int)
balance.drop('Balance', axis=1, inplace=True)

# Merging with original data
LLDB = LLDB.merge(balance, on=['Account_Number', 'month', 'year'])

# Splitting the data into training and testing sets
X = LLDB.drop(['creditworthy', 'Date'], axis=1)
y = LLDB['creditworthy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Making predictions on the testing set and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion matrix:\n", confusion)

# Plotting a confusion matrix
plt.figure(figsize=(5,5))
plt.imshow(confusion, cmap=plt.cm.Blues)
plt.xticks([0,1], ['Not Creditworthy', 'Creditworthy'], rotation=45)
plt.yticks([0,1], ['Not Creditworthy', 'Creditworthy'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.colorbar()
plt.show()

In [ ]:
# Generating predicted probabilities for the positive class using the test set data and trained model
y_prob = model.predict_proba(X_test)[:, 1]

# Computing precision-recall curve
precision, recall, _ = precision_recall_curve(y_test, y_prob)

# Computing area under precision-recall curve
auc_pr = auc(recall, precision)

# Computing ROC curve
fpr, tpr, _ = roc_curve(y_test, y_prob)

# Computing area under ROC curve
auc_roc = auc(fpr, tpr)

# Plotting precision-recall curve
plt.plot(recall, precision, color='blue', label='PR curve (AUC = %0.2f)' % auc_pr)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.show()

# Plotting ROC curve
plt.plot(fpr, tpr, color='red', label='ROC curve (AUC = %0.2f)' % auc_roc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Printing 'LLDB'
LLDB

In [ ]:
# Making predictions on the entire dataset
LLDB['y_pred'] = model.predict(X)

# Grouping the data by customer account number and calculating the difference between actual and predicted values
customer_data = LLDB.groupby('Account_Number').apply(lambda x: (x['creditworthy'] - x['y_pred']).mean())

# Plotting the difference for each customer as a line graph
plt.figure(figsize=(15, 5))
plt.plot(customer_data.index, customer_data.values)
plt.xlabel('Customer Account Number')
plt.ylabel('Difference in Actual and Predicted Values')
plt.title('Creditworthiness Predictions')
plt.show()


# We can see that mostly the difference is around zero but goes to a higher value in some cases. Thus depicting a good accuracy of 82%

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Making predictions on the testing set and evaluate the model
y_predrf = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_predrf)
confusionrf = confusion_matrix(y_test, y_predrf)
print("Accuracy:", accuracy)
print("Confusion matrix:\n", confusion)

# Plotting a confusion matrix
plt.figure(figsize=(5,5))
plt.imshow(confusionrf, cmap=plt.cm.Blues)
plt.xticks([0,1], ['Not Creditworthy', 'Creditworthy'], rotation=45)
plt.yticks([0,1], ['Not Creditworthy', 'Creditworthy'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.colorbar()
plt.show()

# The Randomforest classifier works much better than the Logistic Regressor to predict credit card worthiness

In [ ]:
# Returning unique values of the "Third_Party_Name" column in the "ctob_db" dataframe
ctob_db.Third_Party_Name.unique()

# Categorising expenses and balances based on where the money is spent

In [ ]:
# Defining the categories and their corresponding keywords
categories = {
    "Grocery": ["Tesco", "Coop", "Sainsbury", "Asda", "Morrisons", "Aldi", "Lidl", "Millets"],
    "Food/Drink": ["Adams", "White Hart", "Starbucks", "Coffee", "Red Lion", "Crown", "The Royal Oak" ,"Pizza", "KFC", "Harvester", "Bill's", "McDonald's", "Burger King", "Nando's", "Deliveroo", "Just Eat", "Domino's", "Pizza Hut", "Greggs", "Rose", "Italian"],
    "Fashion": ["Zara", "Matalan", "H&M", "Wool", "Primark", "Gap", "Topshop", "Sports Direct", "Next", "River Island", "Nike", "Adidas", "JD Sports", "Footlocker", "North Face", "TK", "Mountain", "Yarn"],
    "Misc.": ["Amazon", "Ebay", "Shell", "BP", "Esso", "Halifax", "Music", "Netflix", "Now TV", "Spotify", "Pets", "NURSERY", "Hobbycraft", "Gym", "GYM"],
    "Academic/Leisure": ["Waterstones", "Blackwell's" ,"HMV", "WHSmith", "Museum", "Theatre", "Cinema", "National Trust", "Cass Art", "Musical", "CeX", "Game"]
}

In [ ]:
# Creating a new varaibale cb_data and it creates a copy of 'ctob_db' in the new variable 'cb_data'
cb_data = ctob_db.copy()

In [ ]:
# Adding a new column for the transaction category
cb_data["Category"] = ""

# Iterating over the transactions and categorize them
for i, row in cb_data.iterrows():
    for category, keywords in categories.items():
        for keyword in keywords:
            if keyword in row["Third_Party_Name"]:
                cb_data.at[i, "Category"] = category
                break



In [ ]:
# Printing the first 15 transaction data with categories
cb_data.head(15)

In [ ]:
# Checking for empty
cb_data[cb_data['Category'] == ''].index

In [ ]:
# Defining a custom color palette
custom_palette = {"Food/Drink": "blue", "Fashion": "orange", "Academic/Leisure": "green", "Misc.": "red", "Grocery" : "violet"}

# Using the custom color palette with seaborn
sns.set(rc={'figure.figsize':(16,8)})
sns.scatterplot(data=cb_data, x="Balance", y="Amount", hue="Category", palette=custom_palette)
plt.title('Categorizing customers based on sector- wise expenditure')


# Inferences: 
##   1. Least amount of money is spent on misellaneous
##   2. Most amount of moeny is spent on food and drinks. Very few people with negative balance go out for a drink or to a restaurant
##   3. Customers with negative balance rarely spend money on clothes
##   4. Everyone spends money on groceries but it is not that costly. 

# Categorising customers based on the amount they spend and the balance they have

In [ ]:
# Creating a new varaibale CBD and it creates a copy of 'cb_data' in the new variable 'CBD'
CBD = cb_data.copy()

In [ ]:
# Creating a new column for customer category
def categorize_customer(row):
    if row['Amount'] > 1500 and row['Balance'] < 2000:
        return 'High Spender, Low Balance'
    elif row['Amount'] > 1500 and row['Balance'] >= 2000:
        return 'High Spender, High Balance'
    elif row['Amount'] <= 1500 and row['Balance'] < 2000:
        return 'Low Spender, Low Balance'
    else:
        return 'Low Spender, High Balance'

CBD['Customer Category'] = CBD.apply(categorize_customer, axis=1)

# Creating a scatter plot
colors = {'High Spender, Low Balance': 'red', 
          'High Spender, High Balance': 'blue',
          'Low Spender, Low Balance': 'green', 
          'Low Spender, High Balance': 'orange'}

fig, ax = plt.subplots(figsize=(15, 8))
for category, color in colors.items():
    x = CBD[CBD['Customer Category'] == category]['Amount']
    y = CBD[CBD['Customer Category'] == category]['Balance']
    ax.scatter(x, y, c=color, label=category)

ax.legend()
ax.set_xlabel('Amount Spent')
ax.set_ylabel('Balance')
ax.set_title('Customer Spending and Balance')
plt.show()


In [ ]:
# Printing 'cb_data'
cb_data

# Sector Classification using KNN

In [ ]:
# Creating a new varaibale cbd and it creates a copy of 'cb_data' in the new variable 'cbd'
cbd = cb_data.copy()

In [ ]:
# Dropping unwanted columns
cbd = cbd.drop(columns=['Date', 'Time', 'Type_of_Transaction', 'Third_Party_Name'])

# Encoding categorical variables
encoder = LabelEncoder()
cbd['Category'] = encoder.fit_transform(cbd['Category'])

# Splitting the data into training and testing sets
X = cbd.drop(columns=['Category'])
y = cbd['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizing data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Training the KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predicting the categories of the test data
y_pred = knn.predict(X_test)

# Evaluating the model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


# 93% accuracy in sector classification using account number, amount balance and third party account

In [ ]:
# Creating confusion matrix and classification report for model evaluation
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [ ]:
# Plotting a confusion matrix
plt.imshow(cm, cmap=plt.cm.Blues)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.colorbar()
plt.show()

In [ ]:
# Generating a Pandas dataframe from the classification report of the model's performance on the test data
# Removing the 'support' column
cr_df = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T
cr_df.drop(['support'], axis=1, inplace=True)
cr_df.style.background_gradient(cmap='Blues')

# Seasonal Trends for overall spending for customers

In [ ]:
# Printing 'lloyds_group_man'
lloyds_group_mon

In [ ]:
# Grouping the data by 'month_'
monthly_data = lloyds_group_mon.groupby('month_').agg({'Total_Amount': 'sum', 'Number_of_Transaction': 'sum'})

# Plotting the trends
fig, ax1 = plt.subplots()

# Plotting the 'Total Amount Spent'
color = 'tab:red'
ax1.set_xlabel('Month')
ax1.set_ylabel('Total Amount Spent', color=color)
ax1.plot(monthly_data.index, monthly_data['Total_Amount'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Adding a secondary y-axis for the 'Number_of_Transaction'
ax2 = ax1.twinx()

# Plotting the number of transactions
color = 'tab:blue'
ax2.set_ylabel('Number of Transactions', color=color)
ax2.plot(monthly_data.index, monthly_data['Number_of_Transaction'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Adding title and labels
plt.title('Seasonal Trends in Spending')
plt.xticks(monthly_data.index)
plt.show()

# Change in customer spending

In [ ]:
# Aggregating the data by Account_Number and calculate the sum of Total_Amount for each customer
agg_data = lloyds_group_mon.groupby("Account_Number").agg({"Total_Amount": "sum"})

# Calculating the percentage change in spending from the first month to the last month for each customer
agg_data["pct_change"] = (agg_data["Total_Amount"].iloc[-1] - agg_data["Total_Amount"].iloc[0]) / agg_data["Total_Amount"].iloc[0] * 100

# Sorting the customers based on the percentage change in spending
top_inc = agg_data.sort_values("pct_change", ascending=False).head(5)  # top 5 customers with increased spending
top_dec = agg_data.sort_values("pct_change", ascending=True).head(5)   # top 5 customers with decreased spending

# Displaying the results
print("Top 5 customers with increased spending:")
print(top_inc)

print("\nTop 5 customers with decreased spending:")
print(top_dec)

In [ ]:
# Filtering the original data for the top 5 customers with increased spending
inc_data = lloyds_group_mon[lloyds_group_mon["Account_Number"].isin(top_inc.index)]

# Filtering the original data for the top 5 customers with decreased spending
dec_data = lloyds_group_mon[lloyds_group_mon["Account_Number"].isin(top_dec.index)]

# Plotting the 'Total_Amount' against the 'month_' for the top 5 customers with increased spending
for i, grp in inc_data.groupby("Account_Number"):
    plt.plot(grp["month_"], grp["Total_Amount"], label=str(i))
plt.title("Top 5 customers with increased spending")
plt.xlabel("Month")
plt.ylabel("Total Amount")
plt.legend()
plt.show()

# Plotting the 'Total_Amount' against the 'month_' for the top 5 customers with decreased spending
for i, grp in dec_data.groupby("Account_Number"):
    plt.plot(grp["month_"], grp["Total_Amount"], label=str(i))
plt.title("Top 5 customers with decreased spending")
plt.xlabel("Month")
plt.ylabel("Total Amount")
plt.legend()
plt.show()


## Inference  : Bank can approach customer who's spending is low and give them more rewards

# Businesses that pay highest salaries

In [ ]:
# Printing 'ctob_db'
ctob_db

In [ ]:
# Grouping the transactions by 'Third_Party_Name' and sum the 'Amount' column
third_party_totals = ctob_db.groupby('Third_Party_Name')['Amount'].sum().sort_values(ascending=False)

# Printing the top 10 third party names and their total earnings
print(third_party_totals.head(10))

# Businesses that make most money

In [ ]:
# Filtering only the rows where 'Type_of_Transaction' is 'Debit'
df = ctob_db[ctob_db['Type_of_Transaction'] == 'Debit']

# Grouping the data by 'Third_Party_Name' and 'month_', and sum the 'Amount' column for each group
grouped_df = df.groupby(['Third_Party_Name', 'month_'])['Amount'].sum().reset_index()

# Sorting the resulting dataframe in descending order by the total amount spent by customers on each business for all months combined
sorted_df = grouped_df.groupby('Third_Party_Name')['Amount'].sum().reset_index().sort_values('Amount', ascending=True)

# Displaying the top 10 businesses by total amount spent by customers
top_businesses = sorted_df.head(10)
print(top_businesses)


In [ ]:
# Creating a horizontal bar chart for displaying the top 10 performing businesses based on the total negative sum of amounts 
sns.set(style="whitegrid")
plt.figure(figsize=(8,6))
sns.barplot(x='Amount', y='Third_Party_Name', data=top_businesses, palette='viridis')
plt.title('Top 10 Well Performing Businesses', fontsize=16)
plt.xlabel('Total Negative Sum of Amounts', fontsize=12)
plt.ylabel('Business Name', fontsize=12)
plt.show()

# Inference: The bank can offer to pro